## Imports

In [7]:
import math
import numpy  as np
import pandas as pd
import inflection
import seaborn as sns


from scipy                 import stats  as ss
from matplotlib            import pyplot as plt
from IPython.display       import Image
from IPython.core.display  import HTML

from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder


## 5.0 DATA PREPARATION

In [4]:
df5 = pd.read_csv('data/df3_tratado.csv')